In [1]:
%load_ext autoreload
%autoreload 2
from indices import MainDataset
from neighbors import Neighborhoods
import functions
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import Pool
from metric import metrics_stat

In [1]:
import lightgbm

ModuleNotFoundError: No module named 'lightgbm'

In [2]:
train_dataset = MainDataset("dataset/train.csv")
test_dataset = MainDataset("dataset/test.csv", need_index = False)

/Users/ivan.bragin/projects/raif/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
neighborhoods = Neighborhoods(train_dataset.index)
neighborhoods1 = Neighborhoods(train_dataset.index1)

In [4]:
# for i, o in enumerate(train_dataset.all_objects[:10000]):
#     if i % 1000 == 0:
#         print(i)
#     neighbor = neighborhoods.get_haversine_closest(o, 7)
#     for f_name, f in functions.items():
#         prices = [f(o, x[0], x[1]) for x in neighbor]
#         o.neighbors_scores[f_name] = np.mean(prices)

In [5]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(train_dataset.all_objects):
#         o.neighbors_scores = {}
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min
    

253000
254000
255000
256000
253000
254000
255000
256000


In [6]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(test_dataset.all_objects):
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min

0
1000
2000
0
1000
2000


In [7]:
raw_dense_features = [
#     'osm_historic_points_in_0.005',
#     'osm_culture_points_in_0.0075',
        'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
        'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
        'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
                      'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500',
       'total_square', "lat", "lng"
       ]

raw_cat_features = ['city', 'osm_city_nearest_name', 'region', 'realty_type', 'street', 'floor']

bad_f = set(["F_square_False_min", "F_square_False_median"])
def getX(o):
    x = []
    for f_name, score in o.neighbors_scores.items():
        if f_name not in bad_f and "P1_kk" not in f_name:
            x.append(score)
    for n in raw_dense_features:
        x.append(o.row[n])
    for n in raw_cat_features:
        x.append(o.row[n])
    return pd.Series(x)


In [8]:
columns = []
for p in ["",
          "P1_"
         ]:
    for t in ["_median", "_min", "_max"]:
        columns.extend([p + f_name + t for f_name, f in functions.functions.items() ])
columns = [c for c in columns if c not in bad_f]
    # columns = []
columns.extend(raw_dense_features)
columns.extend(raw_cat_features)

In [9]:
# X0 = [getX(o) for o in train_dataset.all_objects[:1] if o.row["price_type"] == 0]
# X0 = pd.concat(X0, axis=1).T
# X0.columns=columns
# Y0 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 0]
# X0["floor"] = X0["floor"].astype("str")

In [10]:
X1 = [getX(o) for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = pd.concat(X1, axis=1).T
X1.columns=columns
Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = X1.fillna(-999)


In [11]:
Xtest1 = [getX(o) for o in test_dataset.all_objects]
Xtest1 = pd.concat(Xtest1, axis=1).T
Xtest1.columns=columns
Xtest1 = Xtest1.fillna(-999)

In [49]:
# X0 = X0.fillna(-999)
X_train, X_val, Y_train, Y_val = train_test_split(X1, Y1, test_size=0.1, random_state=42)

In [13]:
import keras.backend as K

def custom_mse(y_true, y_pred):
 
    # calculating squared difference between target and predicted values 
    loss = K.square(y_pred - y_true)  # (batch_size, 2)
    
    # multiplying the values with weights along batch dimension
    loss = loss * [0.3, 0.7]          # (batch_size, 2)
                
    # summing both loss values along batch dimension 
    loss = K.sum(loss, axis=1)        # (batch_size,)
    
    return loss

ModuleNotFoundError: No module named 'keras'

In [ ]:
def get_input_output(m, s, event_len, mask=True):
    i = L.Input(event_len)
    o = L.Embedding(m+1, s, input_length=event_len, mask_zero=mask)(i)
    return i, o

def get_model():

    
    # 3. числовые фичи как в xgboost
    dense_input = L.Input(len(dense_features))
    x = L.Dense(128, activation="relu")(dense_input)
    
    # объединяем и классифицируем
    x = L.Concatenate()([x,svd,svd1,svd2,svd3,L.Flatten()(clientTypeOut),L.Flatten()(contentTypeOut)])
    x = L.Dense(128, activation="relu")(x)
    x = L.Dense(1, activation="sigmoid")(x)
    return Model([dense_input], x)
model = get_model()
model.summary()